평균 130g으로 알려져있던 상품이 14개의 표본 평균을 계산한 결과 128.451g이 나왔을 때

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)

In [2]:
df = pd.read_csv('./data/ch11_potato.csv')
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [3]:
s_mean = np.mean(sample)
s_mean

128.4507142857143

# 1. 통계적 가설검정

## 1-1. 기본  
상품 집단이 정규분포를 따르고 모평균이 130이고 모분산이 9인 경우, 표본평균 $\bar X \sim N(130, \dfrac {9} {14})$가 $P(\bar X \le x)=0.05$인 $x$ 구하기

In [4]:
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95)

128.68118313069039

- 귀무가설($H_0$) : 차이가 없다, 효과가 없다는 내용을 가진 가설
- 대립가설($H_1$) : 차이가 유의미하다, 효과가 있다는 내용을 가진 가설
- 유의하다(significant) : 드문 값을 얻었을 때, 그것이 우연이 아니라 어떤 의미를 가졌을 때
- 기각역 : 귀무가설이 기각되는 구간
- 채택역 : 귀무가설이 채택되는 구간
- 유의수준 (1%, 5% 등을 사용)
- 임곗값 : 경계선상의 값
- 검정통계량(Z) : 검정에 사용되는 통계량
- p-value : 검정통계량의 절댓값보다 큰 값을 가질 확률  
  
  
예) 유의수준 5%에서, 귀무가설 : 모평균이 130이다 vs 대립가설 : 모평균이 130보다 작다.
- 표본평균이 128.451이 되면 유의하므로 귀무가설을 기각  
- 표본평균이 129가 되면 유의하지 않으므로 귀무가설을 채택
- 검정통계량 : 표본평균
- 임곗값 : 128.681
- p-value < 유의수준이므로 귀무가설을 기각
- 검정통계량 $Z$ < $z_{1-\alpha}$ 이므로 귀무가설을 기각

### 검정통게량

In [5]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

### 임곗값

In [8]:
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

### p-value

In [9]:
rv.cdf(z)

0.026661319523126635

검정통계량의 절댓값이 임곗값의 절댓값보다 크므로 귀무가설 기각  
또는 p-value가 유의수준 5%보다 작으므로 귀무가설 기각

## 1-2. 단측검정과 양측검정

- 단측검정 : 모평균이 x보다 크다/작다
- 양측검정 : 모평균이 x가 아니다

### 양측검정

In [10]:
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

In [11]:
rv.cdf(z) * 2

0.05332263904625327

검정통계량이 채택역에 들어가므로 단측검정 때와 달리 귀무가설을 기각할 수 없음  
또는 p-value가 유의수준 5%보다 크므로 귀무가설을 기각할 수 없음

## 1-3. 오류

### 제1종 오류 (위험률, $\alpha$)
- 귀무가설이 옳을 때, 귀무가설을 기각하는 오류
    - 평균이 130인데도 130이 작다는 결론을 내리는 오류
- 검출하지 말하야 할 것을 검출했으므로 오탐(False Positive)이라고 함

In [12]:
rv = stats.norm(130, 3)

In [13]:
# 제1종 오류를 범하는 비율
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
# 평균이 130, 표준편차가 3인 표본 14개를 뽑아서 평균을 내고, 검정통계량을 계산하여 기각역에 들어가는 비율 계산
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2) 
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.053

### 제2종 오류 (검정력, $1-\beta$)
- 대립가설이 옳을 때, 귀무가설을 채택하는 오류
    - 평균이 130보다 작은데도 130이라는 결론을 내리는 오류
- 검출해야 하는 것을 검출하지 못했으므로 미탐(False Negative)이라고 함
- $\beta$는 분석가가 제어할 수 없는 확률

In [14]:
rv = stats.norm(128, 3)

In [15]:
# 제2종 오류를 범하는 비율
c = stats.norm().isf(0.95)
n_samples = 10000
# 평균이 128, 표준편차가 3인 표본 14개를 뽑아서 평균을 내고, 검정통계량을 계산하여 채택역에 들어가는 비율 계산
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2) 
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
cnt / n_samples

0.250

# 2. 기본적인 가설검정

## 2-1. 정규분포의 모평균 : 모분산을 알고 있는 경우
- 귀무가설 : $\mu = \mu_0$
- 대립가설 : $\mu \ne \mu_0$
- 검정통계량 : $Z = (\bar X-\mu_0)/ \sqrt{\dfrac{\sigma^2}{n}}$
- $|Z| > |z_{\alpha/2}|$ 또는 $|Z| > |z_{1-\alpha/2}|$이면 귀무가설을 기각

In [18]:
def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean - mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
    
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1 - rv.cdf(z)) * 2
    print(f'p-value : {p:.3f}')

In [19]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p-value : 0.053


## 2-2. 정규분포의 모분산
- 귀무가설 : $\sigma^2 = \sigma_0^2$
- 대립가설 : $\sigma^2 \ne \sigma_0^2$
- 검정통계량 : $Y = \dfrac{(n-1)s^2}{\sigma_0^2}$
- $Y > \chi^2_{\alpha/2}(n-1)$ 또는 $Y < \chi^2_{1-\alpha/2}(n-1)$이면 귀무가설을 기각

In [20]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1) * u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
    
    if y < rv.isf(0.5):
        p = rv.cdf(y) * 2
    else:
        p = (1 - rv.cdf(y)) * 2
    print(f'p-value : {p:.3f}')

In [21]:
pvar_test(sample, 9)

귀무가설을 채택
p-value : 0.085


## 2-3. 정규분포의 모평균 : 모분산을 모르는 경우
- 귀무가설 : $\mu = \mu_0$
- 대립가설 : $\mu \ne \mu_0$
- 검정통계량 : $t = (\bar X-\mu_0)/ \sqrt{\dfrac{s^2}{n}}$
- $t > t_{\alpha/2}(n-1)$ 또는 $t < t_{1-\alpha/2}(n-1)$이면 귀무가설을 기각

In [24]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean - mean0) / np.sqrt(u_var / n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
    
    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p-value : {p:.3f}')

In [25]:
pmean_test(sample, 130)

귀무가설을 채택
p-value : 0.169


### `scipy.stats.ttest_1samp` 이용

In [26]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)

# 3. 2표본 문제에 관한 가설검정

|          | 정규분포를 가정할 수 있음 | 정규분포를 가정할 수 없음 |
| -------- | ------------------------- | ------------------------- |
| 대응표본 | 대응비교 t 검정           | 윌콕슨의 부호순위검정     |
| 독립표본 | 독립비교 t 검정           | 만·위트니의 U 검정        |

## 3-1. 대응비교 t 검정
- 대응하는 데이터가 있고, 데이터 차이에 정규분포를 가정할 수 있는 경우 **평균값 차이**에 대한 검정

표본 20명을 상대로 운동 전후 집중력 차이에 대한 검정

In [27]:
training_rel = pd.read_csv('./data/ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head(3)

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68


- 귀무가설 : $\mu_{after} - \mu_{before}=0$
- 대립가설 : $\mu_{after} - \mu_{before}\ne 0$

In [28]:
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head(3)

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13


- 귀무가설 : $\mu_{diff}=0$
- 채택가설 : $\mu_{diff}\ne 0$

### 차이의 평균에 대한 1표본 t검정

In [29]:
t, p = stats.ttest_1samp(training_rel['차'], 0)
p

0.04004419061842953

### 2표본 t검정

In [30]:
t, p = stats.ttest_rel(training_rel['후'], training_rel['전'])
p

0.04004419061842953

p-value가 유의수준 5%보다 작으므로 귀무가설 기각 (운동전후 집중력 차이가 있다)

## 3-2. 독립비교 t 검정
- 대응하는 데이터가 없고 2표본 모집단에 정규분포를 가정할 수 있는 경우 **평균값의 차이**에 대한 검정

B 집단은 운동을 할 때 A 집단의 20명과 B 집단의 20명에 대해 집중력 차이

In [31]:
training_ind = pd.read_csv('./data/ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head(3)

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54


- 귀무가설 : $\mu_{1} - \mu_{2}=0$
- 대립가설 : $\mu_{1} - \mu_{2}\ne 0$
- $X\sim N(\mu_1, \sigma_1^2), \ Y\sim N(\mu_2, \sigma_2^2)$
- 검정통계량 : $t = \dfrac{(\bar X - \bar Y)- (\mu_1-\mu_2)}{\sqrt{\dfrac{s_1^2}{n_1}+\dfrac{s_2^2}{n_2}}}$, $df=\dfrac{\left(\dfrac{s_1^2}{n_1}+\dfrac{s_2^2}{n_2}\right)^2}{\dfrac{s_1^4}{n_1^2(n_1-1)}+\dfrac{s_2^4}{n_2^2(n_2-1)}}$
    - 웰치의 방법

In [33]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var = False)
p

0.08695731107259362

p-value가 유의수준 5%보다 크므로 귀무가설 채택 (운동하는 집단과 집중력에 유의미한 차이가 없다)

## 3-3. 윌콕슨의 부호순위검정
- 대응표본 차이에서 정규분포를 가정할 수 없는 경우 **중앙값의 차이**에 대한 검정

In [35]:
training_rel = pd.read_csv('./data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [36]:
diff = toy_df['후'] - toy_df['전']
toy_df['차'] = diff
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [37]:
# scipy.stats.rankdata 로 절댓값이 작은 것부터 순서대로 순위 부여
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [40]:
# 차이의 부호가 음수인 것과 양수인 것의 순위합
r_minus = np.sum((diff<0)*rank)
r_plus = np.sum((diff>0)*rank)
r_minus, r_plus

(8, 13)

이 때 **더 작은 쪽**이 검정통계량이 되며, 이 값이 임계값보다 작은 경우에 귀무가설 기각하는 단측검정 수행

- 편향이 있을수록 두 값에도 편향이 생기고, 검정통계량은 작은 값이 된다.

In [41]:
# 두 개의 표본을 넣어도 되고
T, p = stats.wilcoxon(training_rel['전'], training_rel['후'])
p

0.03623390197753906

In [42]:
# 전후 차이를 직접 넣어도 된다.
T, p = stats.wilcoxon(training_rel['전'] - training_rel['후'])
p

0.03623390197753906

p-value가 유의수준 5%보다 작으므로 귀무가설 기각 (중앙값의 차이가 유의미하다.)

#### 모집단이 정규분포일 경우에도 사용할 수 있으나 검정력이 낮음

In [43]:
# 차이가 N(3, 16)을 따르고 표본 크기가 20인 10000개의 표본 데이터 생성
n = 10000
diffs = np.round(stats.norm(3,4).rvs(size=(n, 20)))

In [44]:
# 대응비교 t 검정의 검정력
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff, 0)
    if p < alpha:
        cnt +=1
cnt / n

0.883

In [45]:
# 윌콕슨의 부호순위검정의 검정력
cnt = 0
alpha = 0.05
for diff in diffs:
    T, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
cnt / n

C:\Users\master\anaconda3\envs\multi\lib\site-packages\scipy\stats\morestats.py:2967: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


0.873

## 3-4. 만 위트니의 U 검정
- 대응되는 데이터가 없는 2표본 모집단에 정규분포를 가정할 수 없는 경우 **중앙값의 차이**에 대한 검정
    - 윌콕슨의 순위합검정이라고도 함

In [48]:
training_ind = pd.read_csv('./data/ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


데이터 전체에 대해서 값이 작은대로 순위를 부여

In [49]:
rank = stats.rankdata(np.concatenate([toy_df['A'], toy_df['B']]))
rank_df = pd.DataFrame({'A':rank[:5], 'B':rank[5:10]}).astype(int)
rank_df

,A,B
0,3,5
1,6,8
2,1,9
3,10,4
4,2,7


U가 0이든 큰 값이든 편향을 갖고 있으므로 양측검정 수행
- 검정통계량에 **A의 순위합**을 사용 (B의 순위합을 사용해도 상관없음)
- A에 좋은 순위가 몰려있으면 값이 작아지고, 나쁜 순위가 몰려있으면 커지는 것처럼 데이터의 편향을 잘 반영하기 때문
- U 검정의 검정통계량은 **A에 관한 순위합에서 A의 크기를 $n_1$으로 해서 $\dfrac{n_1(n_1+1)}{2}$을 뺀 것**

In [50]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.0

In [51]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'], alternative = 'two-sided')
p

0.05948611166127324

p-value가 유의수준 5%보다 크므로 귀무가설 채택 (유의미한 차이가 없다.)

## 3-5. 카이제곱검정
- 독립성 검정
    - 귀무가설 : X, Y가 독립이다.
    - 대립가설 : X, Y가 독립이 아니다.

- 예시에서는 X는 광고, Y는 상품 구입 비율

광고 A와 B를 한 결과 상품 구입 비율에 유의한 차이가 있는지 검정

In [52]:
ad_df = pd.read_csv('./data/ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


In [53]:
# 분할표
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [54]:
ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [55]:
n_not, n_yes = ad_cross.sum()
n_not, n_yes

(900, 100)

In [56]:
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

- 기대도수 : 두 변수 X, Y가 독립이라고 가정했을 때, 기대되는 도수
- 관측도수 : 실제로 관측된 도수

In [57]:
# 기대도수
ad_ef = pd.DataFrame({'했다':[n_adA * n_yes / n, n_adB * n_yes / n], 
                      '하지 않았다':[n_adA * n_not / n, n_adB * n_not / n]},
                    index = ['A', 'B'])
ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


검정통계량 $Y = \sum_{i} \sum_j \dfrac{\left( O_{ij}-E_{ij}\right)^2}{E_{ij}}$  
$O_{ij}, E_{ij}$는 각각 관측도수와 기대도수의 $i$행 $j$열의 성분  
$Y$는 자유도가 1인 카이제곱분포를 근사적으로 따름

In [58]:
y = ((ad_cross - ad_ef) ** 2 / ad_ef).sum().sum()
y

3.75

In [59]:
rv = stats.chi2(1)
1 - rv.cdf(y)

0.052807511416113395

p-value가 유의수준 5%보다 크므로 귀무가설 채택 (광고의 종류와 구매수는 독립이다, 즉 A와 B의 유의한 차이가 없다)

In [60]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross, correction=False)
chi2, p, dof

(3.75, 0.052807511416113395, 1)

In [61]:
ef

array([[360.,  40.],
       [540.,  60.]])